In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as models
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [127]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
data = torchvision.datasets.MNIST("./", train=True, download=True, transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST("./", train=False, transform=transforms.ToTensor())

In [128]:
train_size = int(0.9*len(data))
test_size = len(data) - train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size,test_size])

BATCH_SIZE = 1000
SHUFFLE = True
LEARNING_RATE = 0.001  

train_loader = DataLoader(train_set, batch_size= BATCH_SIZE, shuffle= SHUFFLE )
test_loader = DataLoader(test_data, batch_size= BATCH_SIZE, shuffle= SHUFFLE)
val_loader = DataLoader(val_set, batch_size= BATCH_SIZE, shuffle=False)

In [129]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 5)
        self.fc1 = nn.Linear(4*4*32, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Implement a train model function so you can re_use it in task 3 and 4. 
# Should return the best performing model after training
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, msg, pretrained):
    best_accuracy = 0
    writer = SummaryWriter()
    for epochs in range(num_epochs):
        for batch_nr, (data, labels) in enumerate(train_loader):
         
            prediction = model.forward(data)

            loss = criterion(prediction, labels)

            loss.backward()

            optimizer.step()

            optimizer.zero_grad()
            writer.add_scalar(msg, loss, epochs)
            print(
            f'\rEpoch {epochs+1} [{batch_nr+1}/{len(train_loader)}] - Loss: {loss}',
            end=''
        )
        val_accuracy = 0
        total = 0
        with torch.no_grad():
            for batch_nr, (data, labels) in enumerate(val_loader):
                prediction = model.forward(data)
                _, predicted = torch.max(prediction, 1)
                val_accuracy += (predicted == labels).sum().item()
                total += labels.size(0)
            print(" ",val_accuracy/total, "Val Acc")
            val_accuracy = val_accuracy/total
            if(best_accuracy < val_accuracy and pretrained == False):
                best_accuracy = val_accuracy
                print("best model here")
                #torch.save(model.state_dict(), "./E2")
def test_model(model, test_loader):
    test_accuracy = 0
    total = 0
    predictList = []
    testList = []
    
    for batch_nr, (data, labels) in enumerate(test_loader):
        prediction = model.forward(data)
        _, predicted = torch.max(prediction, 1)
        for i in range(len(prediction)):
            predictList.append(predicted[i].item())
            testList.append(labels[i].item())
        test_accuracy += (predicted == labels).sum().item()
        total += labels.size(0)
    print(test_accuracy/total, "Test Accuracy")
  

# Hyperparams. Set these to reasonable values
LEARNING_RATE = 0.0001

# Load our network
model = Net()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer

optimizer = optim.Adam(model.parameters(), LEARNING_RATE)

# Train the model
msg = "trainloss leaky relu and adam"
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, 2, msg, pretrained=False)

# Test the model
model.load_state_dict(torch.load("./E2"))
tested_model = test_model(model, test_loader)

Epoch 1 [54/54] - Loss: 1.8351564407348633  0.7475 Val Acc
best model here
Epoch 2 [54/54] - Loss: 0.7468745112419128  0.8426666666666667 Val Acc
best model here
0.8555 Test Accuracy


In [130]:
import ssl
preprocess = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(28),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
ssl._create_default_https_context = ssl._create_unverified_context
data = torchvision.datasets.SVHN("./", split = "train", download=True, transform= preprocess)
test_data = torchvision.datasets.SVHN("./", split= "test",download = True, transform=preprocess)

Using downloaded and verified file: ./train_32x32.mat
Using downloaded and verified file: ./test_32x32.mat


In [131]:
train_size = int(0.9*len(data))
test_size = len(data) - train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size,test_size])

train_loader = DataLoader(train_set, batch_size= BATCH_SIZE, shuffle= SHUFFLE )
test_loader = DataLoader(test_data, batch_size= BATCH_SIZE, shuffle= SHUFFLE)
val_loader = DataLoader(val_set, batch_size= BATCH_SIZE, shuffle=False)

In [118]:
# Test the model
model.load_state_dict(torch.load("./E2"))
model.conv1 = nn.Sequential(
    nn.Conv2d(3,1,1),
    model.conv1
)
#trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, 2, "", pretrained=True) 
tested_model = test_model(model, test_loader)

0.16018746158574063 Test Accuracy


In [125]:
# Fine tuning
model.load_state_dict(torch.load("./E2"))
model.conv1 = nn.Sequential(
    nn.Conv2d(3,1,1),
    model.conv1
)
model.last_linear = nn.Sequential(
    nn.Linear(in_features=100, out_features=100),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=100, out_features=10),
)
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, 2, "", pretrained=True) 
tested_model = test_model(model, test_loader)

Epoch 1 [66/66] - Loss: 2.2239487171173096  0.2022932022932023 Val Acc
Epoch 2 [66/66] - Loss: 2.1577959060668945  0.22904722904722905 Val Acc
0.23839889366933006 Test Accuracy


In [133]:
# Feature Extracting
model.load_state_dict(torch.load("./E2"))
for params in model.parameters():
    params.requires_grad = False

model.conv1 = nn.Sequential(
    nn.Conv2d(3,1,1),
    model.conv1
)
model.last_linear = nn.Sequential(
    nn.Linear(in_features=100, out_features=100),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=100, out_features=10),
)
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, 2, "", pretrained=True) 
tested_model = test_model(model, test_loader)

Epoch 1 [66/66] - Loss: 3.1134521961212167  0.18004368004368004 Val Acc
Epoch 2 [66/66] - Loss: 3.0234601497650146  0.18004368004368004 Val Acc
0.16233866011063305 Test Accuracy
